In [1]:
import math
from types import SimpleNamespace

import wandb
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader, Subset

from utilities import *


In [2]:
# Constants
INPUT_SIZE = 3 * 16 * 16
OUTPUT_SIZE = 5
HIDDEN_SIZE = 256
NUM_WORKERS = 2
CLASSES = ["hero", "non-hero", "food", "spell", "side-facing"]

# Device
device = torch.device("cuda" if torch.cuda.is_available()  else "cpu")
device = torch.device("mps" if torch.backends.mps.is_built() else "cpu")

data_dir = './data/'

def get_dataloaders(batch_size, slice_size=None, valid_pct=0.2):
    dataset = CustomDataset.from_np(data_dir + "sprites_1788_16x16.npy", data_dir + "sprite_labels_nc_1788_16x16.npy", argmax=True)

    if slice_size:
        dataset = dataset.subset(slice_size)

    train_ds, valid_ds = dataset.split(valid_pct)

    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=1)    
    valid_dl = DataLoader(valid_ds, batch_size=batch_size, shuffle=False, num_workers=1)

    return train_dl, valid_dl

def get_model(dropout):
    return nn.Sequential(
        nn.Flatten(),
        nn.Linear(INPUT_SIZE, HIDDEN_SIZE),
        nn.BatchNorm1d(HIDDEN_SIZE),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(HIDDEN_SIZE, OUTPUT_SIZE)
    ).to(device)


In [3]:
train_dl, valid_dl = get_dataloaders(128, slice_size=1000)

sprite shape: (89400, 16, 16, 3)
labels shape: (89400,)
sprite shape: (1000, 16, 16, 3)
labels shape: (1000,)


In [29]:
def validate_model(model, valid_dl, loss_func, log_images=False, batch_idx=0):
    "Compute the performance of the model on the validation dataset and log a wandb.Table"
    model.eval()
    val_loss = 0.0
    correct = 0

    with torch.inference_mode():
        for i, (images, labels) in enumerate(valid_dl):
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            val_loss += loss_func(outputs, labels) * labels.size(0)

            # Compute accuracy and accumulate
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

            # Log one batch of images to the dashboard, always same batch_idx.
            if i == batch_idx and log_images:
                log_image_predictions_table(images, predicted, labels, outputs.softmax(dim=1))

    return val_loss / len(valid_dl.dataset), correct / len(valid_dl.dataset)


def log_image_predictions_table(images, predicted, labels, probs):
    "Create a wandb Table to log images, labels, and predictions"
    table = wandb.Table(columns=["image", "pred", "target"] + [f"score_{i}" for i in range(OUTPUT_SIZE)])
    
    for img, pred, targ, prob in zip(images.cpu(), predicted.cpu(), labels.cpu(), probs.cpu()):
        table.add_data(wandb.Image(img), CLASSES[pred], CLASSES[targ], *prob.numpy())
    
    wandb.log({"predictions_table": table}, commit=False)


In [30]:
def train_model(config):
    "Train a model with a given config"
    wandb.init(
        project="deeplearningai-intro",
        config=config
    )

    # Get the data
    train_dl, valid_dl = get_dataloaders(config.batch_size, config.slice_size, config.valid_pct)
    n_steps_per_epoch = math.ceil(len(train_dl.dataset) / config.batch_size)

    # A simple MLP model
    model = get_model(config.dropout)

    # Make the loss and optimizer
    loss_func = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=config.lr)

    example_ct = 0

    for epoch in tqdm(range(config.epochs), total=config.epochs):
        model.train()

        for step, (images, labels) in enumerate(train_dl):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            train_loss = loss_func(outputs, labels)
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

            example_ct += len(images)
            metrics = {
                "train/train_loss": train_loss,
                "train/epoch": (step + 1 + (n_steps_per_epoch * epoch)) / n_steps_per_epoch,
                "train/example_ct": example_ct
            }

            if step + 1 < n_steps_per_epoch:
                # Log train metrics to wandb 
                wandb.log(metrics)
                
        val_loss, accuracy = validate_model(model, valid_dl, loss_func, log_images=(epoch == (config.epochs - 1)))

        # Log train and validation metrics to wandb
        val_metrics = {
            "val/val_loss": val_loss,
            "val/val_accuracy": accuracy
        }
        wandb.log({**metrics, **val_metrics})

    # If you had a test set, this is how you could log it as a Summary metric
    wandb.run.summary['test_accuracy'] = 0.8

    wandb.finish()


In [31]:
config = SimpleNamespace(
    epochs = 3,
    batch_size = 128,
    lr = 1e-3,
    dropout = 0.1,
    slice_size = 1000,
    valid_pct = 0.2,
)

In [32]:
train_model(config)

sprite shape: (89400, 16, 16, 3)
labels shape: (89400,)
sprite shape: (1000, 16, 16, 3)
labels shape: (1000,)


  0%|          | 0/3 [00:00<?, ?it/s]

train/epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▆▆▆▇▇▇██
train/example_ct,▁▁▂▂▃▃▃▃▄▄▅▅▅▆▆▆▇▇███
train/train_loss,█▆▅▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
val/val_accuracy,▁▆█
val/val_loss,█▃▁
test_accuracy,0.8
train/epoch,3.0
train/example_ct,2400
train/train_loss,0.21717
val/val_accuracy,0.92
val/val_loss,0.32078


Let's try with another value of dropout:

In [28]:
config.dropout = 0.5
train_model(config)

sprite shape: (89400, 16, 16, 3)
labels shape: (89400,)
sprite shape: (1000, 16, 16, 3)
labels shape: (1000,)


  0%|          | 0/3 [00:00<?, ?it/s]

train/epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▆▆▆▇▇▇██
train/example_ct,▁▁▂▂▃▃▃▃▄▄▅▅▅▆▆▆▇▇███
train/train_loss,█▆▅▄▄▃▃▂▂▂▂▁▂▂▁▁▁▁▁▁▁
val/val_accuracy,▁██
val/val_loss,█▂▁
test_accuracy,0.8
train/epoch,3.0
train/example_ct,2400
train/train_loss,0.27209
val/val_accuracy,0.92
val/val_loss,0.32183
